In [ ]:
%pylab inline
import numpy as np
import matplotlib.pyplot as plt
from ovationpyme.ovation_prime import SeasonalFluxEstimator,BinCorrector
from ovationpyme.ovation_utilities import calc_avg_solarwind
import ipywidgets
from collections import OrderedDict
from functools import partial

seasons = ['winter','spring','summer','fall']
auroral_types = ['diff','mono','wave','ions']
flux_types = [
                "electron energy flux",
                "ion energy flux",
                "electron number flux",
                "ion number flux",
                "electron average energy",
                "ion average energy"
            ]

def latlt2polar(lat,lt,hemisphere):
    """
    Converts an array of latitude and lt points to polar for a top-down dialplot (latitude in degrees, LT in hours)
    i.e. makes latitude the radial quantity and MLT the azimuthal 

    get the radial displacement (referenced to down from northern pole if we want to do a top down on the north, 
    or up from south pole if visa-versa)
    """
    from numpy import pi
    if hemisphere=='N':
        r = 90.-lat
    elif hemisphere=='S':
        r = 90.-(-1*lat)
    else:
        raise ValueError('%s is not a valid hemisphere, N or S, please!' % (hemisphere))
    #convert lt to theta (azimuthal angle) in radians
    theta = lt/24. * 2*pi

    #the pi/2 rotates the coordinate system from
    #theta=0 at negative y-axis (local time) to
    #theta=0 at positive x axis (traditional polar coordinates)
    return r,theta

def polar2dial(ax):
    """
    Turns a matplotlib axes polar plot into a dial plot
    """
    #Rotate the plot so that noon is at the top and midnight
    #is at the bottom, and fix the labels so radial direction
    #is latitude and azimuthal direction is local time in hours
    ax.set_theta_zero_location('S')
    theta_label_values = np.array([0.,3.,6.,9.,12.,15.,18.,21.])*180./12
    theta_labels = ['%d:00' % (int(th/180.*12)) for th in theta_label_values.flatten().tolist()]
    ax.set_thetagrids(theta_label_values,labels=theta_labels)

    r_label_values = 90.-np.array([80.,70.,60.,50.])
    r_labels = [r'$%d^{o}$' % (int(90.-rv)) for rv in r_label_values.flatten().tolist()]
    ax.set_rgrids(r_label_values,labels=r_labels)
    ax.set_rlim([0.,40.])

def pcolor_flux(ax,mlatgrid,mltgrid,fluxgrid,hemisphere,**pcolor_kwargs):
    mlats,mlts = mlatgrid.flatten(),mltgrid.flatten()
    flux = fluxgrid.flatten()
    if 'vmin' not in pcolor_kwargs:
        pcolor_kwargs['vmin'] = np.nanpercentile(flux,5)
    if 'vmax' not in pcolor_kwargs:
        pcolor_kwargs['vmax'] = np.nanpercentile(flux,95)
    r,theta = latlt2polar(mlats,mlts,hemisphere)
    rgrid = r.reshape(mlatgrid.shape)
    thetagrid = theta.reshape(mltgrid.shape)
    mappable = ax.pcolormesh(thetagrid,rgrid,fluxgrid,**pcolor_kwargs)
    return mappable

def plot_seasonal_fluxes_callback(clicked_button,widgets):
    f = plt.figure(figsize=(12,6),dpi=150)
    axN = f.add_subplot(121,projection='polar')
    axS = f.add_subplot(122,projection='polar')
    season = widgets['season_select'].value
    atype = widgets['atype_select'].value
    jtype = widgets['jtype_select'].value
    bincorrect = widgets['bincorrect_toggle'].value
    year = widgets['year'].value
    month = widgets['month'].value
    day = widgets['day'].value
    hour = widgets['hour'].value
    minute = widgets['minute'].value
    dt = datetime.datetime(year,month,day,hour,minute)
    avg_sw = calc_avg_solarwind(dt)
    estimator = SeasonalFluxEstimator(season,atype,jtype)
    fluxtuple = estimator.get_gridded_flux(avg_sw['Ec'],combined_N_and_S=False)
    mlatgridN,mltgridN,fluxgridN,mlatgridS,mltgridS,fluxgridS = fluxtuple
    if bincorrect:
        bcN = BinCorrector(mlatgridN,mltgridN)
        fluxgridN = bcN.fix(fluxgridN)
        bcS = BinCorrector(mlatgridS,mltgridS)
        fluxgridS = bcS.fix(fluxgridS)
        print("Correction Applied")    
    pcolor_kwargs = {'cmap':'viridis'}
    manual_vmin = wgts['vmin_float'].value
    if manual_vmin != -1.:
        pcolor_kwargs['vmin']=manual_vmin
    manual_vmax = wgts['vmax_float'].value
    if manual_vmax != -1.:
        pcolor_kwargs['vmax']=manual_vmax
    mappableN = pcolor_flux(axN,mlatgridN,mltgridN,fluxgridN,'N',**pcolor_kwargs)
    mappableS = pcolor_flux(axS,mlatgridS,mltgridS,fluxgridS,'S',**pcolor_kwargs)
    f.colorbar(mappableN,ax=axN,label=jtype)
    f.colorbar(mappableS,ax=axS,label=jtype)
    axN.set_title('Northern Hemisphere')
    axS.set_title('Southern Hemisphere')
    polar2dial(axN)
    polar2dial(axS)
    f.suptitle('Seasonal Flux %s, %s, %s\nNewell Coupling: %f' % (season,atype,jtype,avg_sw['Ec']))
    plt.show()
    

wgts = OrderedDict()
wgts['year']=ipywidgets.IntText(description='Year',value=2010)
wgts['month']=ipywidgets.IntText(description='Month',value=5)
wgts['day']=ipywidgets.IntText(description='Day',value=29)
wgts['hour']=ipywidgets.IntText(description='Hour',value=12)
wgts['minute']=ipywidgets.IntText(description='Minute',value=0)
wgts['season_select']=ipywidgets.Dropdown(options=seasons,description='Season')
wgts['atype_select']=ipywidgets.Dropdown(options=auroral_types,description='Type of Aurora')
wgts['jtype_select']=ipywidgets.Dropdown(options=flux_types,description='Type of Flux')
wgts['vmin_float']=ipywidgets.FloatText(description='Colorbar Min (-1 is Autoscale)',value=-1)
wgts['vmax_float']=ipywidgets.FloatText(description='Colorbar Max (-1 is Autoscale)',value=-1)
wgts['bincorrect_toggle']=ipywidgets.ToggleButton(description='Correction',value=False)
wgts['run_button'] = ipywidgets.Button(description='Calculate')
wgts['run_button'].on_click(partial(plot_seasonal_fluxes_callback,widgets=wgts))

for widget_name,widget in wgts.iteritems():
    display(widget)